In [ ]:
from google.colab import drive
from glob import glob
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, AveragePooling2D, Dense, Activation, DepthwiseConv2D
from tensorflow.keras import layers
from tensorflow.keras.layers import Flatten

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip "/content/gdrive/MyDrive/BreaKHis_400X.zip"

Archive:  /content/gdrive/MyDrive/BreaKHis_400X.zip
  inflating: BreaKHis 400X/test/benign/SOB_B_A-14-22549AB-400-001.png  
  inflating: BreaKHis 400X/test/benign/SOB_B_A-14-22549AB-400-003.png  
  inflating: BreaKHis 400X/test/benign/SOB_B_A-14-22549AB-400-004.png  
  inflating: BreaKHis 400X/test/benign/SOB_B_A-14-22549AB-400-005.png  
  inflating: BreaKHis 400X/test/benign/SOB_B_A-14-22549AB-400-007.png  
  inflating: BreaKHis 400X/test/benign/SOB_B_A-14-22549AB-400-011.png  
  inflating: BreaKHis 400X/test/benign/SOB_B_A-14-22549AB-400-015.png  
  inflating: BreaKHis 400X/test/benign/SOB_B_A-14-22549AB-400-019.png  
  inflating: BreaKHis 400X/test/benign/SOB_B_A-14-22549AB-400-025.png  
  inflating: BreaKHis 400X/test/benign/SOB_B_A-14-22549AB-400-027.png  
  inflating: BreaKHis 400X/test/benign/SOB_B_A-14-22549AB-400-028.png  
  inflating: BreaKHis 400X/test/benign/SOB_B_A-14-22549CD-400-004.png  
  inflating: BreaKHis 400X/test/benign/SOB_B_A-14-22549CD-400-006.png  
  inflating:

In [ ]:
# Paths to the dataset
Training = glob('/content/BreaKHis 400X/train*/*')
Testing = glob('/content/BreaKHis 400X/test*/*')

In [ ]:
def load_and_preprocess_data(paths, label):
    data = []  #store the image data
    labels = []# store corresponding labels


    for path in paths:
        img = cv2.imread(path)
        img = cv2.resize(img, (224, 224))
        data.append(img)
        labels.append(label)

    return np.array(data), np.array(labels)

In [ ]:
# training dataset
train_benign_paths = glob('/content/BreaKHis 400X/train/benign*/*')
train_malignant_paths = glob('/content/BreaKHis 400X/train/malignant*/*')

# Load and preprocess the training data
train_data_benign, train_labels_benign = load_and_preprocess_data(train_benign_paths, 0)  # Label 0 for benign
train_data_malignant, train_labels_malignant = load_and_preprocess_data(train_malignant_paths, 1)  # Label 1 for malignant

# Combine the training data and labels
train_data = np.concatenate((train_data_benign, train_data_malignant), axis=0)
train_labels = np.concatenate((train_labels_benign, train_labels_malignant), axis=0)

# Convert labels to numerical values
lb = LabelEncoder()
train_labels_encoded = lb.fit_transform(train_labels)

# One-hot encode the training labels >>>>  used to represent categorical variables as binary vectors
train_labels_one_hot = to_categorical(train_labels_encoded)

# Partition the training data into training and validation sets
trainX, valX, trainY, valY = train_test_split(train_data, train_labels_one_hot, test_size=0.2, random_state=42, stratify=train_labels_one_hot)

In [ ]:
#  testing dataset
test_benign_paths = glob('/content/BreaKHis 400X/test/benign*/*')
test_malignant_paths = glob('/content/BreaKHis 400X/test/malignant*/*')

# Load and preprocess the test data
test_data_benign, test_labels_benign = load_and_preprocess_data(test_benign_paths, 0)  # Label 0 for benign
test_data_malignant, test_labels_malignant = load_and_preprocess_data(test_malignant_paths, 1)  # Label 1 for malignant

# Combine the test data and labels
test_data = np.concatenate((test_data_benign, test_data_malignant), axis=0)
test_labels = np.concatenate((test_labels_benign, test_labels_malignant), axis=0)

# Convert labels to numerical values
test_labels_encoded = lb.transform(test_labels)

# One-hot encode the test labels
test_labels_one_hot = to_categorical(test_labels_encoded)

In [ ]:
class SqueezeExcite(tf.keras.layers.Layer):# to perform channel-wise feature recalibration or attention, enhancing important features while suppressing less relevant ones

    def __init__(self, input_channels, ratio=4):
        super(SqueezeExcite, self).__init__()
        self.pooling = tf.keras.layers.GlobalAveragePooling2D()
        self.fc1 = tf.keras.layers.Dense(input_channels // ratio, activation='relu')# ratio>reduces the dimensionality of the input by dividing the number of input channels
        self.fc2 = tf.keras.layers.Dense(input_channels, activation='sigmoid')#to produce channel-wise attention weights (between 0 and 1) for each channel

    def call(self, inputs, training=None, mask=None):
        x = self.pooling(inputs)
        x = self.fc1(x)
        x = self.fc2(x)
        x = tf.keras.layers.Reshape((1, 1, -1))(x)
        return inputs * x


In [ ]:
class BottleNeck(tf.keras.layers.Layer):#contributes to feature extraction with a reduced number of parameters
    def __init__(self, in_size, exp_size, out_size, s, is_se_existing, NL, k):
        super(BottleNeck, self).__init__()
        self.is_se_existing = is_se_existing
        self.stride = s
        self.in_size = in_size
        self.out_size = out_size
        self.exp_size = exp_size
        self.NL = NL
        self.k = k

        self.conv1 = Conv2D(filters=self.exp_size, kernel_size=(1, 1), strides=1, padding="same")
        self.bn1 = BatchNormalization()

        self.conv2 = Conv2D(filters=self.exp_size, kernel_size=(3, 3), strides=self.stride, padding="same", groups=self.exp_size)
        self.bn2 = BatchNormalization()

        self.se = tf.keras.Sequential([
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(self.exp_size // 4, activation='relu'),# reduces the number of units (neurons) to a fraction of the original size
                                        # //4>>>balance between reducing the number of parameters and allowing the network to capture meaningful information
            tf.keras.layers.Dense(self.exp_size, activation='sigmoid')#(0 or 1) 0>>channel is less important
                                                                      #learning the relationships between channels &
                                                                      #determining how much each channel should be emphasized or suppressed
        ]) if self.is_se_existing else None

        self.conv3 = Conv2D(filters=self.out_size, kernel_size=(1, 1), strides=1, padding="same")
        self.bn3 = BatchNormalization()

    def call(self, inputs, training=False):
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = h_swish(x)

        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = h_swish(x)

        if self.is_se_existing:
            se_tensor = tf.reshape(self.se(x), [-1, 1, 1, self.exp_size])
            x = x * se_tensor

        x = self.conv3(x)
        x = self.bn3(x, training=training)

        if self.stride == 1 and self.in_size == self.out_size:
            x = x + inputs


        return x



In [ ]:
# hard swish activation
#designed to introduce non-linearity to the network while ensuring a smoother transition between zero and the linear region
def h_swish(x):
    return x * tf.nn.relu6(x + 3) / 6.0

In [ ]:
class MobileNetV3Small(tf.keras.Model):
    def __init__(self, num_classes, final_activation='softmax'):
        super(MobileNetV3Small, self).__init__()
        self.conv1 = Conv2D(filters=16, kernel_size=(3, 3), strides=2, padding="same")
        self.bn1 = BatchNormalization()
        self.bneck1 = BottleNeck(in_size=16, exp_size=16, out_size=16, s=2, is_se_existing=True, NL="RE", k=3)


        self.conv2 = Conv2D(filters=576, kernel_size=(1, 1), strides=1, padding="same")
        self.bn2 = BatchNormalization()
        self.avgpool = AveragePooling2D(pool_size=(7, 7), strides=1)
        self.flatten = Flatten()  # converts the input from a multidimensional tensor into a one-dimensional tensor.
        self.output_layer = Dense(num_classes, activation=final_activation, name='output')

    def call(self, inputs, training=False):
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = h_swish(x)

        x = self.bneck1(x, training=training)

        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = h_swish(x)
        x = self.avgpool(x)
        x = self.flatten(x)
        output = self.output_layer(x)
        return output

In [ ]:
# Instantiate the model
num_classes = 2
model = MobileNetV3Small(num_classes, final_activation='softmax')

# model summary
model.build((None, 224, 224, 3))  # input shape is (224, 224, 3)
model.summary()

Model: "mobile_net_v3_small"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  448       
                                                                 
 batch_normalization (Batch  multiple                  64        
 Normalization)                                                  
                                                                 
 bottle_neck (BottleNeck)    multiple                  1044      
                                                                 
 conv2d_4 (Conv2D)           multiple                  9792      
                                                                 
 batch_normalization_4 (Bat  multiple                  2304      
 chNormalization)                                                
                                                                 
 average_pooling2d (Average  multiple          

In [ ]:
model = Sequential()
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# separate directories for training and validation
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/BreaKHis 400X/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'  ,
    shuffle=False
)

validation_generator = validation_datagen.flow_from_directory(
    '/content/BreaKHis 400X/test',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',
    shuffle=False
)

Found 1148 images belonging to 2 classes.
Found 545 images belonging to 2 classes.


In [ ]:
# Calculate class weights
class_weights = {
    0: len(train_labels_one_hot) / (2 * np.bincount(train_labels_encoded)[0]),
    1: len(train_labels_one_hot) / (2 * np.bincount(train_labels_encoded)[1])
}

In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])
# Train the model with validation data
epochs = 30
history = model.fit(
    train_generator,
    epochs=epochs,
    verbose=1,
    validation_data=validation_generator,
    class_weight=class_weights
)

Epoch 1/30
36/36 [==============================] - 30s 797ms/step - loss: 0.7069 - accuracy: 0.6768 - val_loss: 0.5415 - val_accuracy: 0.7358
Epoch 2/30
36/36 [==============================] - 28s 778ms/step - loss: 0.5564 - accuracy: 0.7875 - val_loss: 1.0634 - val_accuracy: 0.3651
Epoch 3/30
36/36 [==============================] - 25s 701ms/step - loss: 0.7500 - accuracy: 0.5218 - val_loss: 0.8554 - val_accuracy: 0.4477
Epoch 4/30
36/36 [==============================] - 28s 785ms/step - loss: 0.6561 - accuracy: 0.6908 - val_loss: 0.9979 - val_accuracy: 0.4000
Epoch 5/30
36/36 [==============================] - 29s 803ms/step - loss: 0.6686 - accuracy: 0.7639 - val_loss: 1.6271 - val_accuracy: 0.3321
Epoch 6/30
36/36 [==============================] - 29s 811ms/step - loss: 0.7808 - accuracy: 0.5697 - val_loss: 0.5446 - val_accuracy: 0.7780
Epoch 7/30
36/36 [==============================] - 28s 783ms/step - loss: 0.5837 - accuracy: 0.6821 - val_loss: 0.6679 - val_accuracy: 0.6771